In [31]:
import numpy as np
import scipy.spatial.distance as dist
import matplotlib.pyplot as plt
import librosa as lb
import os.path
from pathlib import Path
import pickle
import multiprocessing
import time
import gc
from numba import jit, njit, cuda, float64
import torch

In [4]:
DTWDefaultSteps = np.array([[1, 1, 2],
                            [1, 2, 1]], dtype = np.uint32)

DTWDefaultWeights = np.array([2, 3, 3], dtype = np.float64)


subseqDTWDefaultSteps = np.array([[1, 1, 2],
                                  [1, 2, 1]], dtype = np.uint32)

subseqDTWDefaultWeights = np.array([1, 1, 2], dtype = np.float64)


MAX_FLOAT = float('inf')

In [5]:
@njit
def DTW_Cost_To_DandB(C, Steps = DTWDefaultSteps, weights = DTWDefaultWeights, subsequence=False):
    '''
    Find the accumulated cost matrix and backtrace matrix from a cost matrix using Dynamic time warping
    
    Arguments:
    C -- The Cost matrix
    Steps -- The available steps, where the first row is the row steps, the second row is the column steps
    weights -- The weights of the steps
    subsequence -- True if subsequence DTW should be performed rather than standard DTW
             
    Returns:
    D -- The accumulated cost matrix
    B -- The backtrace matrix
    '''
    '''
    Section for verifying input
    '''
    # Ensure parameters have correct dtypes and dimensions
    try:
        C = C.astype(np.float64)
        assert C.ndim == 2
    except:
        print("FAILURE: The type of the cost matrix is wrong - please pass in a 2-d numpy array")
        return

    try:
        Steps = Steps.astype(np.uint32)
        assert Steps.ndim == 2
    except:
        print("FAILURE: The type of the steps matrix is wrong - please pass in a 2-d numpy array")
        return

    try:
        weights = weights.astype(np.float64)
        assert weights.ndim == 1
    except:
        print("FAILURE: The type of the steps matrix is wrong - please pass in a 2-d numpy array")
        return
    
    # Ensure steps and weights sizes are appropriate
    stepsShape = Steps.shape
    numDifSteps = len(weights)
    
    if stepsShape[0] != 2:
        print("FAILURE: The size of the steps matrix is wrong - please pass in a 2-d numpy array with 2 rows")
        return
    if stepsShape[1] != numDifSteps:
        print("FAILURE: The number of steps and weights do not line up - please make sure that there are the same number of steps and weights")
        return
    
    '''
    Algorithm
    '''
    # Separate steps
    rowSteps = Steps[0,:]
    colSteps = Steps[1,:]

    # Define Relevant Variables
    numRows = C.shape[0]
    numCols = C.shape[1]
    
    numDifSteps = len(weights)
    maxRowStep = max(rowSteps)
    maxColStep = max(colSteps)
    
    # Set up accumulated cost matrix D and backtrace matrix B
    D = np.ones((numRows + maxRowStep, numCols + maxColStep), dtype = np.float64) * MAX_FLOAT
    B = np.zeros((numRows, numCols), dtype = np.uint32)
    
    # Fill up D and B
    if subsequence:  # Initialize entire bottom row of D for subsequence
        D[maxRowStep, maxColStep:] = C[0,:]
    else:
        D[maxRowStep, maxColStep] = C[0,0]  # Initialize bottom corner if for standard DTW
    for row in range(maxRowStep, numRows + maxRowStep, 1):
        for col in range(maxColStep, numCols + maxColStep, 1):
            bestCost = D[row, col]
            bestCostIndex = 0
            # Go through each step, find the best one
            for stepIndex in range(numDifSteps):
                costForStep = D[row - rowSteps[stepIndex], col - colSteps[stepIndex]] + weights[stepIndex] * C[row - maxRowStep, col - maxColStep]
                if costForStep < bestCost:
                    bestCost = costForStep
                    bestCostIndex = stepIndex
            # Save best cost and step
            D[row,col] = bestCost
            B[row - maxRowStep, col - maxColStep] = bestCostIndex
    
    # Return accumulated cost matrix D and backtrace matrix B
    return D[maxRowStep:, maxColStep:], B

In [6]:
@njit
def DTW_Backtrace(D, B, Steps=DTWDefaultSteps, subsequence=False, startCol=-1):
    '''
    Backtrace through an accumulated cost matrix and backtrace matrix to find a path
    
    Arguments:
    D -- The accumulated cost matrix
    B -- The backtrace matrix
    Steps -- The available steps
    subsequence -- True if subsequence DTW should be performed rather than standard DTW
    startCol -- The column to begin backtracing from, or -1 if not specified
    
    Returns:
    fwdPath -- A 2d numpy array storing the optimal path. The first row is the path through the rows.
            The second row is the path through the columns
    '''
    '''
    Section for verifying input
    '''
    # Ensure parameters have correct dtypes and dimensions
    try:
        D = D.astype(np.float64)
        assert D.ndim == 2
    except:
        print("FAILURE: The type of the accumulated cost matrix is wrong - please pass in a 2-d numpy array")
        return

    try:
        B = B.astype(np.uint32)
        assert B.ndim == 2
    except:
        print("FAILURE: The type of the backtrace matrix is wrong - please pass in a 2-d numpy array")
        return
    
    try:
        Steps = Steps.astype(np.uint32)
        assert Steps.ndim == 2
    except:
        print("FAILURE: The type of the steps matrix is wrong - please pass in a 2-d numpy array")
        return

    # Ensure that D and B are the same shape
    if D.shape != B.shape:
        print("FAILURE: The accumulated cost matrix and backtrace matrix are not the same size - please make sure their sizes match")
        return
    
    
    '''
    Backtrace through D and B
    '''
    # Separate steps
    rowSteps = Steps[0,:]
    colSteps = Steps[1,:]
    
    # Initialize variables
    numRows = D.shape[0]
    numCols = D.shape[1]
    
    curRow = numRows - 1  # Always start at last row
    curCol = numCols - 1  # Standard DTW: Start at top-right corner
    if startCol > 0:
        curCol = startCol
    elif subsequence:  # Subsequence: Choose lowest cost of top row
        curCol = np.argmin(D[numRows-1,:])
    
    endCol = curCol
    endCost = D[curRow, curCol]
    stepsInPath = 1
    stepIndex = 0
    done = (subsequence and curRow == 0) or (curRow == 0 and curCol == 0)
    path = np.zeros((2, numRows + numCols), dtype=np.uint32) # make as large as could need, then chop at the end
    path[0, 0] = curRow
    path[1, 0] = curCol
    
    # Backtrace
    while not done:
        if D[curRow, curCol] == MAX_FLOAT:  # No path exists to current location
            return
        
        # you're done if you've made it to the bottom left (non sub-sequence)
        # or just the bottom (sub-sequence)
        # find the step size
        curStepIndex = B[curRow, curCol]
        curRowStep = rowSteps[curStepIndex]
        curColStep = colSteps[curStepIndex]
        # backtrack by 1 step
        curRow = curRow - curRowStep
        curCol = curCol - curColStep
        # add your new location onto the path
        path[0, stepsInPath] = curRow
        path[1, stepsInPath] = curCol
        stepsInPath = stepsInPath + 1
        # check to see if you're done
        done = (subsequence and curRow == 0) or (curRow == 0 and curCol == 0)
        
    # reverse the path (a matrix with two rows) and return it
    fwdPath = np.fliplr(path[:, 0:stepsInPath])
    return fwdPath
    #return fwdPath, endCol, endCost

In [7]:
@njit
def getSegmentEndingLocs(wp):
    '''
    Takes in a segment level path through and returns the columns where each segment ends
    
    Arguments:
    wp -- A segment level path (as a 2d numpy array with 2 rows)
    
    Returns:
    endLocs -- A list where the ith entry is the column where the ith segment ends
    '''
    prevLoc = wp[:,0] # [r,c]
    endLocs = []
    for i in range(wp.shape[1]):
        curLoc = wp[:,i]
        if curLoc[0] != prevLoc[0]: # if row changes
            endLocs.append(curLoc[1])
        prevLoc = curLoc
        
    return endLocs

In [54]:
@cuda.jit
def GPU_DTW_shared(D, B, C, segLength):
    '''
    '''
    # Extract segments into shared memory
    i = cuda.grid(1)
    
    # init shared matrices
    #C_frag = cuda.shared.array((segLength,C.shape[1]), float64)
    #D_frag = cuda.shared.array((segLength,C.shape[1]), float64)
    C_shared = cuda.shared.array(shape=(segLength, C.shape[1], 2), dtype=float64)
    
    # fill matrices
    for r in range(C_shared.shape[0]):
        for c in range(C.shape[1]):
            C_shared[r,c,0] = C[int(min(i*segLength+r, C.shape[0])), c]
            C_shared[r,c,1] = D[int(min(i*segLength+r, C.shape[0])), c]
    
    #C_shared[:,:,0] = C[i*segLength:min((i+1)*segLength, C.shape[1]),:]
    #C_shared[:,:,1] = D[i*segLength:min((i+1)*segLength, C.shape[1]),:]
    
    #C_frag = C[i*segLength:min((i+1)*segLength, C.shape[1]),:]
    #D_frag = D[i*segLength:min((i+1)*segLength, C.shape[1]),:]
    
    # Run DTW
    
    # init steps/weights
    steps = np.array([[1, 1, 2],
                      [1, 2, 1]], dtype = np.uint32)

    weights = np.array([1, 1, 2], dtype = np.float64)
    
    # Separate steps
    rowSteps = steps[0,:]
    colSteps = steps[1,:]

    # Define Relevant Variables
    numRows = C_shared.shape[0]
    numCols = C_shared.shape[1]
    
    numDifSteps = len(weights)
    maxRowStep = max(rowSteps)
    maxColStep = max(colSteps)
    
    # Fill up D and B
    for c in range(C_shared.shape[1]): # subsequence init
        C_shared[0,c,1] = C_shared[0,c,0] 
    
    for row in range(numRows):
        for col in range(numCols):
            bestCost = C_shared[row, col, 1]
            bestCostIndex = 0
            # Go through each step, find the best one
            for stepIndex in range(numDifSteps):
                
                if row - rowSteps[stepIndex] < 0 or col - colSteps[stepIndex] < 0:
                    continue
                
                costForStep = C_shared[row - rowSteps[stepIndex], col - colSteps[stepIndex], 1] \
                            + weights[stepIndex] * C_shared[row, col, 0]
                
                if costForStep < bestCost:
                    bestCost = costForStep
                    bestCostIndex = stepIndex
            # Save best cost and step
            C_shared[row,col,1] = bestCost
            B[row + i*segLength, col] = bestCostIndex
    
    # Copy D and B segments into global memory
    for r in range(C_shared.shape[0]):
        if (i*segLength + r == C.shape[1]):
            break
        for c in range(C_shared.shape[1]):
            D[i*segLength+r, c] = C_shared[r,c,1]

In [48]:
def GPU_WSDTW(queryFeatureFile, refFeatureFile, K, Steps = subseqDTWDefaultSteps, weights = subseqDTWDefaultWeights):
    '''
    '''
    
    # Extract Feature matrices
    F1 = np.load(queryFeatureFile, allow_pickle = True)
    F2 = np.load(refFeatureFile, allow_pickle = True)
    
    F1 = F1[:100, :100]
    F2 = F2[:100, :100]
    
    if max(F1.shape[1], F2.shape[1]) / min(F1.shape[1], F2.shape[1]) >= 2: # no valid path possible
        print("FAILURE - Feature sizes are too different to align with DTW")
        return
    
    # Compute Cost
    # NOTE: This could also be done segment by segment.
    # I am doing it all up front to match the implementation in the old code.
    C = 1 - F1.T @ F2
    
    # Subsequence DTW each segment without backtracing
    segLength = int(np.ceil(F1.shape[1] / K))
    Cseg = np.zeros((K + 1, C.shape[1]), dtype = np.float64)
    D = np.ones(C.shape, dtype = np.float64) * MAX_FLOAT
    B = np.zeros(C.shape, dtype = np.int32)
    totalSize = C.nbytes + D.nbytes + B.nbytes
    totalSegmentSize = int(np.ceil((totalSize-B.nbytes) / K))  # If necessary, maybe just put C and D in shared memory
    if totalSize > torch.cuda.get_device_properties(0).total_memory:  # Pass in fragment by fragment
        print('Fragment too large for global memory')
        # Pass in as many fragments as possible at once
        # Some Function
    elif totalSegmentSize > 96000:  # Can't use shared memory
        print('Fragment too large for shared memory')
        print(totalSegmentSize)
        # Some Function
    else:
        #Some Function
        GPU_DTW_shared[K,1,0,(segLength, C.shape[1], 2)](D, B, C, segLength)
        
    # run segment-level DTW (Not subsequence)
    segmentSteps = np.array([[0, 1],
                             [1, C.shape[0]//(2 * segments)]], #NOTE: This could cause differences with old implementation
                            dtype=np.uint32)
    segmentWeights = np.array([0, 1])
    
    Dseg, Bseg = DTW_Cost_To_DandB(Cseg, Steps = segmentSteps, weights = segmentWeights)
    
    wpseg = DTW_Backtrace(Dseg, Bseg, Steps=segmentSteps)
    
    # Backtrace through segments with segment level path as guide
    # On GPU?
    path = []
    # Frame level backtrace segment by segment
    segmentEndIdxs = getSegmentEndingLocs(wpseg)
    for i, endidx in enumerate(segmentEndIdxs):
        pathSeg = DTW_Backtrace(Dparts[i], Bparts[i], Steps = Steps, subsequence = True, startCol = endidx)
        # Add offset to row indices so they match with overall path
        pathSeg[0,:] = pathSeg[0,:] + (i * segLength)

        # Append fragment to full path
        path.append(pathSeg.copy())
    
    wp_merged = np.hstack(path)
    
    if outfile:
        pickle.dump(wp_merged, open(outfile, 'wb'))

    if profile:
        return wp_merged, np.diff(times)
    else:
        return wp_merged

In [11]:
featureFile1 = "implementation_test/test_features/Features_1.npy"
featureFile2 = "implementation_test/test_features/Features_2.npy"
featureFile3 = "implementation_test/test_features/Features_3.npy"
featureFile4 = "implementation_test/test_features/Features_4.npy"

In [55]:
GPU_WSDTW(featureFile3, featureFile4, 5, Steps = subseqDTWDefaultSteps, weights = subseqDTWDefaultWeights)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<function shared.array at 0x7f1d1cbbc320>) found for signature:
 
 >>> array(dtype=class(float64))
 
There are 2 candidate implementations:
[1m      - Of which 2 did not match due to:
      Overload in function 'array': File: numba/cuda/cudadecl.py: Line 44.
        With argument(s): '(dtype=class(float64))':[0m
[1m       Rejected as the implementation raised a specific error:
         TypeError: typer() missing 1 required positional argument: 'shape'[0m
  raised from /home/khoe/anaconda3/envs/MIR/lib/python3.7/site-packages/numba/core/typing/templates.py:399
[0m
[0m[1mDuring: resolving callee type: Function(<function shared.array at 0x7f1d1cbbc320>)[0m
[0m[1mDuring: typing of call at <ipython-input-54-55c86e763aed> (11)
[0m
[1m
File "<ipython-input-54-55c86e763aed>", line 11:[0m
[1mdef GPU_DTW_shared(D, B, C, segLength):
    <source elided>
    #D_fr

In [13]:
t = torch.cuda.get_device_properties(0).total_memory
t

11721506816